In [ ]:
# Q106_A · Tiny multilayer robustness experiment (effective layer)
# Single-cell Colab version. Fully offline, no API keys required.

import math
import random
import textwrap

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


# ============================================================
# Intro and high-level description
# ============================================================

def print_intro():
    lines = """
    Q106_A · Tiny multilayer robustness (effective layer)

    This notebook is fully offline. No API key is required.

    It will:
      1) Build two families of tiny multilayer networks:
         - Design R: loosely coupled, more robust style.
         - Design F: tightly coupled, fragile style.
      2) Simulate random and targeted attacks at different damage levels.
      3) Propagate cross layer cascades until the system stabilizes.
      4) Compute a scalar robustness tension T_robust for each design.
      5) Save a CSV of raw results and a couple of PNG plots.

    The goal is to make "hidden fragility" visible as a simple tension observable.
    For the broader WFGY Tension Universe context see:
      https://github.com/onestardao/WFGY
    """
    print(textwrap.dedent(lines))


# ============================================================
# Global configuration
# ============================================================

# Number of nodes in each layer
N_NODES = 200

# Edge probabilities for Design R (loosely coupled random graphs)
P_POWER_R = 0.035
P_COMM_R = 0.035

# Number of hubs for Design F (tightly coupled hub-and-spoke)
NUM_HUBS = 5

# Attack settings
ATTACK_FRACTIONS = [0.05, 0.10, 0.20]    # 5%, 10%, 20% of nodes
ATTACK_TYPES = ["random", "targeted"]    # random vs degree-based targeted

# How many independent runs per (design, attack_type, attack_fraction)
RUNS_PER_CASE = 20

# Internal failure thresholds (minimum degree in each layer)
MIN_DEG_POWER = 1
MIN_DEG_COMM = 1

# Base seed so results are reproducible
BASE_SEED = 106


# ============================================================
# Helper functions: build multilayer networks
# ============================================================

def make_design_r_graphs(seed: int):
    """
    Build power and communication layers for Design R.

    Design R is "more robust style":
      - Both layers are Erdos-Renyi random graphs with moderate edge density.
      - Degrees are more evenly spread, no extreme hubs.
    """
    rng = np.random.RandomState(seed)
    power_seed = int(rng.randint(0, 1_000_000))
    comm_seed = int(rng.randint(0, 1_000_000))

    G_power = nx.erdos_renyi_graph(N_NODES, P_POWER_R, seed=power_seed)
    G_comm = nx.erdos_renyi_graph(N_NODES, P_COMM_R, seed=comm_seed)

    return G_power, G_comm


def make_hub_graph(n: int, num_hubs: int, seed: int):
    """
    Build a simple hub-and-spoke style graph for Design F.

    Steps:
      1) Choose a small set of hub nodes.
      2) Connect hubs into a clique.
      3) Attach every non-hub to at least one hub.
      4) Optionally add extra edges to hubs and some leaf-to-leaf edges.
    """
    rng = np.random.RandomState(seed)
    hubs = list(range(num_hubs))

    G = nx.Graph()
    G.add_nodes_from(range(n))

    # Step 1: connect hubs as a clique
    for i in range(num_hubs):
        for j in range(i + 1, num_hubs):
            G.add_edge(hubs[i], hubs[j])

    # Step 2: attach non-hub nodes mainly to hubs
    for node in range(num_hubs, n):
        # connect to one main hub
        hub = int(rng.choice(hubs))
        G.add_edge(node, hub)

        # optional extra connection to another hub
        if rng.rand() < 0.5:
            other = int(rng.choice(hubs))
            if other != hub:
                G.add_edge(node, other)

        # small chance to connect to another non-hub
        if rng.rand() < 0.15:
            other_leaf = int(rng.randint(num_hubs, n))
            if other_leaf != node:
                G.add_edge(node, other_leaf)

    return G


def make_design_f_graphs(seed: int):
    """
    Build power and communication layers for Design F.

    Design F is "fragile style":
      - Both layers are hub-heavy.
      - Many nodes depend on a small hub set.
      - This should be vulnerable to targeted attacks on hubs.
    """
    rng = np.random.RandomState(seed)
    power_seed = int(rng.randint(0, 1_000_000))
    comm_seed = int(rng.randint(0, 1_000_000))

    G_power = make_hub_graph(N_NODES, NUM_HUBS, power_seed)
    G_comm = make_hub_graph(N_NODES, NUM_HUBS, comm_seed)

    return G_power, G_comm


# ============================================================
# Helper functions: cascade and observables
# ============================================================

def largest_component_nodes(G: nx.Graph, alive_nodes: set):
    """
    Return the node set of the largest connected component
    in the subgraph induced by alive_nodes.
    """
    if not alive_nodes:
        return set()

    sub = G.subgraph(alive_nodes)
    if sub.number_of_nodes() == 0:
        return set()

    components = list(nx.connected_components(sub))
    if not components:
        return set()

    largest = max(components, key=len)
    return set(largest)


def compute_internal_failures(G: nx.Graph, alive_nodes: set, min_deg: int):
    """
    Layer-internal failure rule.

    A node fails in this layer if:
      - It is not in the largest connected component, OR
      - Its degree in the alive-node subgraph is below min_deg.
    """
    if not alive_nodes:
        return set()

    sub = G.subgraph(alive_nodes)
    if sub.number_of_nodes() == 0:
        return set()

    gcc_nodes = largest_component_nodes(G, alive_nodes)
    fails = set()

    for node in alive_nodes:
        in_gcc = node in gcc_nodes
        deg = sub.degree(node) if node in sub else 0
        if (not in_gcc) or (deg < min_deg):
            fails.add(node)

    return fails


def run_cascade(G_power: nx.Graph,
                G_comm: nx.Graph,
                design: str,
                initial_alive: set):
    """
    Run cross-layer cascade until it stabilizes.

    Design R cross-layer rule:
      - Node is removed from the system only if it fails in BOTH layers.

    Design F cross-layer rule:
      - Node is removed if it fails in EITHER layer.

    Returns:
      final_alive: nodes that remain after cascades.
      cascade_removed: nodes that were removed by cascades
                       (not counting the initial attack).
    """
    alive = set(initial_alive)
    cascade_removed = set()

    while True:
        if not alive:
            break

        power_fails = compute_internal_failures(G_power, alive, MIN_DEG_POWER)
        comm_fails = compute_internal_failures(G_comm, alive, MIN_DEG_COMM)

        if design == "R":
            candidate = power_fails & comm_fails
        elif design == "F":
            candidate = power_fails | comm_fails
        else:
            raise ValueError(f"Unknown design: {design}")

        new_fail = candidate & alive
        if not new_fail:
            break

        alive -= new_fail
        cascade_removed |= new_fail

    return alive, cascade_removed


def compute_service_metrics(G_power: nx.Graph,
                            G_comm: nx.Graph,
                            alive_nodes: set):
    """
    Compute service observables for the final state.

    Returns:
      S_power: fraction of all nodes in the power-layer giant component.
      S_comm:  fraction of all nodes in the comm-layer giant component.
      S_multi: fraction of all nodes that are in BOTH layer giants.
    """
    if not alive_nodes:
        return 0.0, 0.0, 0.0

    alive = set(alive_nodes)

    sub_power = G_power.subgraph(alive)
    sub_comm = G_comm.subgraph(alive)

    if sub_power.number_of_nodes() == 0:
        S_power = 0.0
        power_gcc_nodes = set()
    else:
        power_components = list(nx.connected_components(sub_power))
        power_gcc_nodes = max(power_components, key=len)
        S_power = len(power_gcc_nodes) / N_NODES

    if sub_comm.number_of_nodes() == 0:
        S_comm = 0.0
        comm_gcc_nodes = set()
    else:
        comm_components = list(nx.connected_components(sub_comm))
        comm_gcc_nodes = max(comm_components, key=len)
        S_comm = len(comm_gcc_nodes) / N_NODES

    multi_nodes = set(power_gcc_nodes) & set(comm_gcc_nodes)
    S_multi = len(multi_nodes) / N_NODES

    return S_power, S_comm, S_multi


# ============================================================
# Attack generation
# ============================================================

def choose_attacked_nodes(G_power: nx.Graph,
                          design_seed: int,
                          attack_type: str,
                          attack_fraction: float):
    """
    Choose initial attacked nodes.

    - random:  uniform sample of nodes.
    - targeted: top-degree nodes in the power layer (hub attack).
    """
    num_attack = max(1, int(round(N_NODES * attack_fraction)))

    rng = random.Random(design_seed + hash((attack_type, attack_fraction)) % 1_000_000)
    all_nodes = list(range(N_NODES))

    if attack_type == "random":
        attacked = set(rng.sample(all_nodes, num_attack))
    elif attack_type == "targeted":
        degrees = G_power.degree()
        degree_list = sorted(degrees, key=lambda x: x[1], reverse=True)
        attacked = {node for node, _deg in degree_list[:num_attack]}
    else:
        raise ValueError(f"Unknown attack_type: {attack_type}")

    return attacked


# ============================================================
# Robustness target and T_robust
# ============================================================

def robustness_target(attack_fraction: float) -> float:
    """
    Declared robustness target S_target as a function of attack level.

    Interpretation:
      - Up to 5% damage: claim is "keep at least 80% service".
      - Up to 10% damage: claim is "keep at least 60% service".
      - Up to 20% damage: claim is "keep at least 40% service".
    """
    if attack_fraction <= 0.05:
        return 0.80
    if attack_fraction <= 0.10:
        return 0.60
    return 0.40


def compute_T_robust(summary_df: pd.DataFrame):
    """
    Compute scenario-level violations and a design-level T_robust score.

    Steps:
      1) For each scenario, compute S_target from attack_fraction.
      2) Set S_multi_mean = S_multi_final_mean (for readability).
      3) violation = max(S_target - S_multi_mean, 0).
      4) T_robust for each design = average violation over all scenarios.
    """
    df = summary_df.copy()

    df["S_target"] = df["attack_fraction"].apply(robustness_target)
    df["S_multi_mean"] = df["S_multi_final_mean"]
    df["violation"] = (df["S_target"] - df["S_multi_mean"]).clip(lower=0.0)

    design_scores = (
        df.groupby("design")["violation"]
        .mean()
        .reset_index()
        .rename(columns={"violation": "T_robust"})
    )

    return df, design_scores


# ============================================================
# Main experiment loop
# ============================================================

def run_experiment():
    print_intro()

    rows = []

    total_cases = len(["R", "F"]) * len(ATTACK_TYPES) * len(ATTACK_FRACTIONS) * RUNS_PER_CASE
    case_counter = 0

    # Main simulation loop
    for design in ["R", "F"]:
        for attack_type in ATTACK_TYPES:
            for attack_fraction in ATTACK_FRACTIONS:
                for run_id in range(RUNS_PER_CASE):
                    # Use different seeds for different runs and designs
                    network_seed = BASE_SEED + 1000 * run_id + (0 if design == "R" else 500_000)

                    # Build multilayer network
                    if design == "R":
                        G_power, G_comm = make_design_r_graphs(network_seed)
                    else:
                        G_power, G_comm = make_design_f_graphs(network_seed)

                    # Choose initial attack set
                    attacked = choose_attacked_nodes(
                        G_power=G_power,
                        design_seed=network_seed,
                        attack_type=attack_type,
                        attack_fraction=attack_fraction,
                    )

                    # Alive nodes after the initial attack
                    initial_alive = set(range(N_NODES)) - attacked

                    # Run cross-layer cascade
                    final_alive, cascade_removed = run_cascade(
                        G_power=G_power,
                        G_comm=G_comm,
                        design=design,
                        initial_alive=initial_alive,
                    )

                    # Compute service observables
                    S_power, S_comm, S_multi = compute_service_metrics(
                        G_power=G_power,
                        G_comm=G_comm,
                        alive_nodes=final_alive,
                    )

                    # Fractions for reporting
                    cascade_fraction = len(cascade_removed) / N_NODES
                    initial_fraction = len(attacked) / N_NODES

                    rows.append(
                        {
                            "design": design,
                            "attack_type": attack_type,
                            "attack_fraction": attack_fraction,
                            "run_id": run_id,
                            "S_power_final": S_power,
                            "S_comm_final": S_comm,
                            "S_multi_final": S_multi,
                            "initial_removed_fraction": initial_fraction,
                            "cascade_removed_fraction": cascade_fraction,
                        }
                    )

                    case_counter += 1
                    if case_counter % 20 == 0:
                        print(f"  Progress: {case_counter}/{total_cases} runs completed...")

    # Convert to DataFrame and save raw results
    df_raw = pd.DataFrame(rows)
    csv_path = "Q106_A_results.csv"
    df_raw.to_csv(csv_path, index=False)
    print()
    print(f"Saved raw results to: {csv_path}")

    # --------------------------------------------------------
    # Aggregated summary
    # --------------------------------------------------------
    summary_cols = [
        "S_power_final",
        "S_comm_final",
        "S_multi_final",
        "initial_removed_fraction",
        "cascade_removed_fraction",
    ]
    summary = (
        df_raw.groupby(["design", "attack_type", "attack_fraction"])[summary_cols]
        .agg(["mean", "std"])
        .reset_index()
    )

    # Flatten MultiIndex columns
    summary.columns = [
        "_".join(col).rstrip("_") if isinstance(col, tuple) else col for col in summary.columns
    ]

    print()
    print("Aggregated summary (per design x attack_type x attack_fraction):")
    print(summary.head(12))

    # Compute T_robust
    df_for_tension, design_scores = compute_T_robust(summary)

    print()
    print("Design-level robustness tension T_robust (smaller is better):")
    print(design_scores)

    # --------------------------------------------------------
    # Plot 1: S_multi vs attack_fraction (for random and targeted attacks)
    # --------------------------------------------------------
    print()
    print("Generating plot: Q106_A_service_vs_attack.png...")

    fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

    for idx, attack_type in enumerate(ATTACK_TYPES):
        ax = axes[idx]
        sub = df_for_tension[df_for_tension["attack_type"] == attack_type]

        for design in ["R", "F"]:
            s = sub[sub["design"] == design]
            x = s["attack_fraction"]
            y = s["S_multi_mean"]
            yerr = s["S_multi_final_std"]
            ax.errorbar(
                x,
                y,
                yerr=yerr,
                marker="o",
                linestyle="-",
                label=f"Design {design}",
            )

        ax.set_title(f"{attack_type} attack")
        ax.set_xlabel("attack fraction")
        if idx == 0:
            ax.set_ylabel("S_multi (service level)")
        ax.set_ylim(-0.05, 1.05)
        ax.grid(True)
        ax.legend()

    fig.suptitle("Q106_A · Multilayer service vs attack level")
    plt.tight_layout()
    plot1_path = "Q106_A_service_vs_attack.png"
    plt.savefig(plot1_path, dpi=150)
    plt.close(fig)

    print(f"Saved figure: {plot1_path}")

    # --------------------------------------------------------
    # Plot 2: cascade breakdown for targeted 10% attack
    # --------------------------------------------------------
    print("Generating plot: Q106_A_cascade_examples.png...")

    example_fraction = 0.10
    example_type = "targeted"
    example = df_for_tension[
        (df_for_tension["attack_type"] == example_type)
        & (df_for_tension["attack_fraction"] == example_fraction)
    ]

    if not example.empty:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

        for idx, design in enumerate(["R", "F"]):
            ax = axes[idx]
            row = example[example["design"] == design]
            if row.empty:
                continue

            initial_mean = float(row["initial_removed_fraction_mean"])
            cascade_mean = float(row["cascade_removed_fraction_mean"])
            S_multi_mean = float(row["S_multi_mean"])

            # Survived fraction after initial attack and cascades
            survived = max(0.0, 1.0 - initial_mean - cascade_mean)

            labels = ["initial", "cascade", "survived"]
            values = [initial_mean, cascade_mean, survived]

            ax.bar(labels, values)
            ax.set_ylim(0.0, 1.0)
            ax.set_title(f"Design {design}\n{example_type} attack, f={example_fraction}")
            ax.set_ylabel("fraction of nodes")

        fig.suptitle("Q106_A · Example cascades (targeted attack 10%)")
        plt.tight_layout()
        plot2_path = "Q106_A_cascade_examples.png"
        plt.savefig(plot2_path, dpi=150)
        plt.close(fig)
        print(f"Saved figure: {plot2_path}")
    else:
        print("No matching scenario found for cascade example plot.")

    print()
    print("Done. See Q106_A_results.csv and the PNG files in the Colab workspace.")
    print("For the full WFGY 3.0 Tension Universe pack, visit:")
    print("  https://github.com/onestardao/WFGY")


# ============================================================
# Entry point (single-cell one-button run)
# ============================================================

def main():
    run_experiment()


if __name__ == "__main__":
    main()
